# Testing Spherical Harmonics and Other Components

In [17]:
# This is just giving me incorrect results, I'm pretty sure
using BenchmarkTools
using Test
using SphericalHarmonics

include("Spherical.jl");

In [5]:
# Testing memoization
@benchmark cg(1, 1, 1, -1, 2, 0)

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):  15.516 ns … 725.125 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     17.417 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.605 ns ±  14.290 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▂▆█▆▅▃                                                     ▂
  ▄████████▇▇▇▆▇▅▇▆▆▆▄▅▄▄▄▃▃▃▄▄▄▄▃▁▃▅▅▅▅▅▅▅▄▄▅▄▆▁▅▅▃▅▃▃▁▃▅▃▄▅▅ █
  15.5 ns       Histogram: log(frequency) by time      40.9 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [7]:
@testset "Clebsch-Gordan Coefficients" begin
    # Some examples from Wikipedia
    # TODO Get SymPy.jl comparison to work

    # cg(j1, m1, j2, m2, J, M) = <j1 m1 j2 m2; J M>
    @test cg(1, 1, 1, -1, 2, 0) == √(1 / 6)
    @test cg(1, 0, 1, 0, 2, 0) == √(2 / 3)
    @test cg(1, 0, 1, 0, 0, 0) == -√(1 / 3)
    @test cg(1, 0, 1, 0, 1, 0) == 0

    @test cg(2, 1, 1, 0, 3, 1) == √(8 / 15)
end;

Test Summary:               | Pass  Total  Time
Clebsch-Gordan Coefficients |    5      5  0.0s


In [49]:
@testset "Spherical Harmonics" begin
    Y00(θ, ϕ) = 0.5 * sqrt(1/π)
    Y1m1(θ, ϕ) = 0.5 * sqrt(3/(2*π))*sin(θ)*exp(-im*ϕ)
    Y10(θ, ϕ) = 0.5 * sqrt(3/π)*cos(θ)
    Y11(θ, ϕ) = -0.5 * sqrt(3/(2*π))*sin(θ)*exp(im*ϕ)

    # Real version
    function Y1mR(θ, ϕ, m::Integer)
        ans = Y10(θ, ϕ) |> real
        if m > 0
            # TODO Check proper minus signs, make sure convention is consistent
            ans = -(Y1m1(θ, ϕ) + (-1)^m * Y11(θ, ϕ)) / √2 |> real
        elseif m < 0
            ans = -im * (Y1m1(θ, ϕ) - (-1)^m * Y11(θ, ϕ)) / √2 |> real
        end

        return ans
    end

    n_samples = 10
    for n = 1:n_samples
        θ = rand() * π
        ϕ = (rand()-0.5) * 2*π
        YsR = computeYlm(θ, ϕ; lmax=1, SHType=SphericalHarmonics.RealHarmonics())

        @test YsR[(0, 0)] ≈ Y00(θ, ϕ)
        @test YsR[(1, -1)] ≈ Y1mR(θ, ϕ, -1)
        @test YsR[(1, 1)] ≈ Y1mR(θ, ϕ, 1)
        @test YsR[(1, 0)] ≈ Y1mR(θ, ϕ, 0)
        #@test Ys[(1, -1)] ≈ Y1m1(θ, ϕ)
    end
end

Test Summary:       | Pass  Total  Time
Spherical Harmonics |   40     40  0.0s


Test.DefaultTestSet("Spherical Harmonics", Any[], 40, false, false, true, 1.672838802067e9, 1.672838802094e9)